In [12]:
%matplotlib inline
import inspect, os, sys, copy, pytz, re, glob
import simplejson as json
import pandas as pd
from dateutil import parser
import datetime
import matplotlib.pyplot as plt   # Matplotlib for plotting
import matplotlib.dates as md
import numpy as np
import seaborn as sns
from collections import Counter, defaultdict

utc=pytz.UTC

ENV = "production"
os.environ['CS_ENV'] = 'production'
BASE_DIR = "/home/reddit/CivilServant"
sys.path.append(BASE_DIR)
subreddit_id = "2qh13"

with open(os.path.join(BASE_DIR, "config") + "/{env}.json".format(env=ENV), "r") as config:
  DBCONFIG = json.loads(config.read())

### LOAD SQLALCHEMY
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text, and_, or_
from app.models import Base, SubredditPage, FrontPage, Subreddit, Post, ModAction, Experiment
from utils.common import PageType

db_engine = create_engine("mysql://{user}:{password}@{host}/{database}".format(
    host = DBCONFIG['host'],
    user = DBCONFIG['user'],
    password = DBCONFIG['password'],
    database = DBCONFIG['database']))
DBSession = sessionmaker(bind=db_engine)
db_session = DBSession()

### LOAD PRAW
import reddit.connection
conn = reddit.connection.Connect(base_dir=BASE_DIR, env="jupyter")

### FILTER OUT DEPRECATION WARNINGS ASSOCIATED WITH DECORATORS
# https://github.com/ipython/ipython/issues/9242
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, message='.*use @default decorator instead.*')

In [13]:
earliest_date = db_session.query(Experiment).filter(Experiment.id==8).first().created_at

### Coding Plan
1. get experiment posts
2. get all posts from subreddit (confirm that this is at least all that appeared in the rankings) (purpose: to determine the creation time of the post)
3. run through the list of pages, page by page, and create the per-post ranking snapshot dataset, without having to store every subreddit HOT page in memory

### Collect All Subreddit posts

In [14]:
# tabloid_domains = [ 'dailymail.co.uk', 'express.co.uk','mirror.co.uk', 'news.com.au', 
#                     'nypost.com', 'thesun.co.uk','dailystar.co.uk','metro.co.uk']
# msm_domains     = [ "bbc.com", "reuters.com", "nytimes.com", "washingtonpost.com", "cnn.com", 
#                     "telegraph.co.uk", "latimes.com", "huffingtonpost.com", "theguardian.com", "forbes.com",
#                     "examiner.com", "usatoday.com", "wsj.com", "cbsnews.com", "cbc.ca", "time.com",
#                     "sfgate.com", "newsweek.com", "bostonglobe.com", "nydailynews.com", "msnbc.com",
#                     "foxnews.com", "aljazeera.com", "nbcnews.com", "npr.org", "bloomberg.com", "abcnews.com", 
#                     "aljazeera.com", "bigstory.ap.com", "cbc.ca", "time.com"]

all_posts = {}
for row in db_engine.execute(text("select * from posts WHERE subreddit_id = '2qh13' ORDER BY posts.created ASC;")):
    post = {}
    post_data = json.loads(row['post_data'])
    for key in post_data.keys():
        post[key] = post_data[key]
    for key in row.keys():
        post[key]=row[key]
    del post['post_data']
    all_posts[post['id']] = post['created']

### Process Subreddit Pages

In [33]:
#pages = db_session.query(SubredditPage).filter(and_(SubredditPage.page_type == PageType.HOT,
#                                                    SubredditPage.created_at >= earliest_date)).limit(1)

included_posts = set()
excluded_posts = set()
for page in db_engine.execute(text("select * from subreddit_pages WHERE subreddit_id='2qh13' AND page_type={0} AND created_at > '{1}' LIMIT 5;".format(
    4, #PageType.HOT
    earliest_date
))):
    posts = json.loads(page['page_data'])
    for post in posts:
        if(post['id'] in all_posts.keys()):
            included_posts.add(post['id'])
        else:
            excluded_posts.add(post['id'])

In [38]:
page['created_at'] - 

{'author': 'Brianlife',
 'id': '5f016v',
 'num_comments': 83,
 'num_reports': 0,
 'score': 250,
 'subreddit_id': 't5_2qh13'}

### Conclusion: Many Posts are Excluded, So I Should Plan to Query reddit for Creation Date Info

In [24]:
list(all_posts.values())[0]

{'approved_by': None,
 'archived': False,
 'author': 'god_im_bored',
 'author_flair_css_class': None,
 'author_flair_text': None,
 'banned_by': None,
 'clicked': False,
 'comment_data': None,
 'comments_queried_at': None,
 'contest_mode': False,
 'created': datetime.datetime(2016, 11, 2, 6, 36, 7),
 'created_at': datetime.datetime(2016, 11, 2, 10, 36, 25),
 'created_utc': 1478082967.0,
 'distinguished': None,
 'domain': 'timesofisrael.com',
 'downs': 0,
 'edited': False,
 'gilded': 0,
 'hidden': False,
 'hide_score': False,
 'id': '5ap5oo',
 'is_self': False,
 'likes': None,
 'link_flair_css_class': 'palestisrael',
 'link_flair_text': 'Israel/Palestine',
 'locked': False,
 'media': None,
 'media_embed': {},
 'mod_reports': [],
 'name': 't3_5ap5oo',
 'num_comments': 0,
 'num_reports': None,
 'over_18': False,
 'permalink': '/r/worldnews/comments/5ap5oo/eu_declares_israel_boycott_protected_as_free/',
 'quarantine': False,
 'removal_reason': None,
 'report_reasons': None,
 'saved': False,